In [1]:
!pip install transformers torch torchvision pandas tqdm scikit-learn pillow bert-score evaluate rouge-score rapidfuzz sentence-transformers inflect

  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 70.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.1/61.1 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 84.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 10.7 MB/s eta 0:00:00
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=2493

In [2]:
from PIL import Image
import torch
from tqdm import tqdm
from transformers import BlipProcessor, BlipForQuestionAnswering
import pandas as pd
from pathlib import Path
from sklearn.preprocessing import LabelEncoder
from torchvision import models, transforms
from bert_score import score
import re
import inflect
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
from bert_score import BERTScorer
from rouge_score import rouge_scorer
from rapidfuzz.distance import Levenshtein
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

2025-05-11 18:28:54.143159: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1746988134.315226      19 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1746988134.364305      19 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
df = pd.read_csv("/kaggle/input/finaldataset/finalDataset.csv")
df['answer'] = df['answer'].astype(str)
df = df.dropna(subset=['question', 'answer'])

In [4]:
df

,image_id,full_image_path,question,answer
0,81+4dBN1jsL,abo-images-small/images/small\9d/9dfccb37.jpg,What shape is this?,Rectangular
1,81+4dBN1jsL,abo-images-small/images/small\9d/9dfccb37.jpg,Is it a cover?,Yes
2,81+4dBN1jsL,abo-images-small/images/small\9d/9dfccb37.jpg,What color is bottom?,Red
3,81+4dBN1jsL,abo-images-small/images/small\9d/9dfccb37.jpg,Are there multiple colors?,Yes
4,81+4dBN1jsL,abo-images-small/images/small\9d/9dfccb37.jpg,Is there a camera hole?,Yes
...,...,...,...,...
159145,81yoY30yFiL,abo-images-small/images/small\7a/7ab6a11b.jpg,What word is centered on the cover?,Super
159146,81yoY30yFiL,abo-images-small/images/small\7a/7ab6a11b.jpg,What color is the written text?,Yellow
159147,81yoY30yFiL,abo-images-small/images/small\7a/7ab6a11b.jpg,What is the shape of the cover?,Rectangle
159148,81yoY30yFiL,abo-images-small/images/small\7a/7ab6a11b.jpg,What is the background comprised of?,Fragments


In [5]:
answer_choices = df["answer"].dropna().unique().tolist()
df["full_image_path"] = df["full_image_path"].apply(lambda p: p.replace("\\", "/"))
df["full_image_path"] = df["full_image_path"].apply(lambda p: str(Path("/kaggle/input") / p))

In [6]:
print("Yes:",df["answer"].value_counts()['Yes'])
print("No:",df["answer"].value_counts()['No'])

Yes: 64883
No: 7595


In [7]:
import pandas as pd

# Normalize the answer column to match "yes" and "no" correctly
def normalize_answer(ans):
    ans = str(ans).strip().lower()
    return ans

df['normalized_answer'] = df['answer'].apply(normalize_answer)

# Split the DataFrame
yes_no_df = df[df['normalized_answer'].isin(['yes', 'no'])]
non_yes_no_df = df[~df['normalized_answer'].isin(['yes', 'no'])]

# Sample
yes_no_sampled = yes_no_df.sample(n=14000, random_state=42)
non_yes_no_sampled = non_yes_no_df.sample(n=86659, random_state=42)

# Combine and shuffle
balanced_df = pd.concat([yes_no_sampled, non_yes_no_sampled]).sample(frac=1, random_state=42).reset_index(drop=True)

# Optional: drop the extra column
balanced_df = balanced_df.drop(columns=['normalized_answer'])

# Use balanced_df in your evaluation


In [8]:
df = balanced_df
df['full_image_path'].iloc[0]

'/kaggle/input/abo-images-small/images/small/0c/0c972760.jpg'

In [9]:
print("Yes:",df["answer"].value_counts()['Yes'])
print("No:",df["answer"].value_counts()['No'])

Yes: 12523
No: 1476


In [10]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processorBLIP = BlipProcessor.from_pretrained("Salesforce/blip-vqa-base")
modelBLIP = BlipForQuestionAnswering.from_pretrained("Salesforce/blip-vqa-base").to(device)


Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


preprocessor_config.json:   0%|          | 0.00/445 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/592 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/4.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.54G [00:00<?, ?B/s]

In [11]:
correct_BLIP = 0
total_BLIP = 0
all_preds_BLIP = []
all_actuals_BLIP = []

In [12]:
import re
import inflect
inflect_engine = inflect.engine()

def normalize_answer(ans):
    ans = ans.strip().lower()
    ans = re.sub(r'[^\w\s]', '', ans)  # remove punctuation
    if ans.isdigit():
        ans = inflect_engine.number_to_words(ans)
    words = ans.split()
    return words[0] if words else ""  # return first word or empty string



In [13]:
for idx, row in tqdm(df.iterrows(), total = len(df)):
    image_path = Path(row['full_image_path'])
    if not image_path.exists():
        continue

    try:
        image = Image.open(image_path).convert("RGB")
    except:
        continue

    question = row['question']
    actual_answer = normalize_answer(row['answer'])

    try:
        inputs = processorBLIP(image, question, return_tensors="pt").to(device)

        with torch.no_grad():
            output_ids = modelBLIP.generate(
                **inputs,
                max_new_tokens=5,
                do_sample=False
            )
            pred_raw = processorBLIP.tokenizer.decode(output_ids[0], skip_special_tokens=True)
            pred = normalize_answer(pred_raw)

    except Exception as e:
        continue
    if not pred:
        continue

    all_preds_BLIP.append(pred)
    all_actuals_BLIP.append(actual_answer)

    if pred == actual_answer:
        correct_BLIP += 1
    total_BLIP += 1

    #print(f"Q: {question} | GT: {actual_answer} | Pred: {pred}")
    if idx % 10000 == 0:
        print(f"Processed {idx}. Correct: {correct_BLIP}/{total_BLIP}")


  0%|          | 2/100659 [00:00<11:16:42,  2.48it/s]

Processed 0. Correct: 0/1


 10%|▉         | 10002/100659 [15:08<2:13:55, 11.28it/s]

Processed 10000. Correct: 5011/9997


 20%|█▉        | 20003/100659 [30:07<1:56:02, 11.58it/s]

...

 89%|████████▉ | 90003/100659 [2:11:14<14:56, 11.88it/s]

Processed 90000. Correct: 44780/89967


 99%|█████████▉| 100003/100659 [2:25:15<00:55, 11.81it/s]

Processed 100000. Correct: 49812/99963


100%|██████████| 100659/100659 [2:26:11<00:00, 11.48it/s]


In [14]:
accuracy = correct_BLIP / total_BLIP
print(f"\nViLT Accuracy: {accuracy:.2f} ({correct_BLIP}/{total_BLIP})")


ViLT Accuracy: 0.50 (50137/100621)


In [15]:
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
import evaluate
from bert_score import BERTScorer
from rouge_score import rouge_scorer
from rapidfuzz.distance import Levenshtein
from sentence_transformers import SentenceTransformer, util
import numpy as np
import pandas as pd

In [16]:


# Normalize case
preds_l = [p.lower() for p in all_preds_BLIP]
refs_l = [r.lower() for r in all_actuals_BLIP]

# Compute exact-match binary metrics
y_pred_bin = [int(p == r) for p, r in zip(preds_l, refs_l)]
y_true_bin = [1] * len(all_actuals_BLIP)

acc = accuracy_score(y_true_bin, y_pred_bin)
prec, rec, f1, _ = precision_recall_fscore_support(
    y_true_bin, y_pred_bin, average="binary", zero_division=0
)

# Print metrics
print(f"Exact-match Accuracy: {acc:.3f}")
print(f"Exact-match Precision: {prec:.3f}")
print(f"Exact-match Recall:    {rec:.3f}")
print(f"Exact-match F1:        {f1:.3f}\n")

# Save predictions and ground truths to CSV
# pred_ref_df = pd.DataFrame({
#     "Prediction": predictions,
#     "Ground_Truth": refs
# })
# df.to_csv('prediction_output.csv', index=False)

Exact-match Accuracy: 0.498
Exact-match Precision: 1.000
Exact-match Recall:    0.498
Exact-match F1:        0.665



In [17]:
from bert_score import score

P, R, F1 = score(all_preds_BLIP, all_actuals_BLIP, lang="en", verbose=True)
print(f"BERTScore - Precision: {P.mean():.4f}, Recall: {R.mean():.4f}, F1: {F1.mean():.4f}")

tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/482 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

...

calculating scores...
computing bert embedding.


  0%|          | 0/38 [00:00<?, ?it/s]

computing greedy matching.


  0%|          | 0/1573 [00:00<?, ?it/s]

done in 16.56 seconds, 6077.59 sentences/sec
BERTScore - Precision: 0.9910, Recall: 0.9881, F1: 0.9894


In [18]:
scorer = rouge_scorer.RougeScorer(['rouge1', 'rougeL'], use_stemmer=True)
rouge1_scores = []
rougeL_scores = []

for pred, ref in zip(preds_l, refs_l):
    scores = scorer.score(ref, pred)
    rouge1_scores.append(scores["rouge1"].fmeasure)
    rougeL_scores.append(scores["rougeL"].fmeasure)

print(f"ROUGE Scores:")
print(f"ROUGE-1 F1: {np.mean(rouge1_scores):.4f}")
print(f"ROUGE-L F1: {np.mean(rougeL_scores):.4f}")

ROUGE Scores:
ROUGE-1 F1: 0.5057
ROUGE-L F1: 0.5057


In [19]:
lev_dists = [Levenshtein.distance(p, r) for p, r in zip(preds_l, refs_l)]
avg_lev = np.mean(lev_dists)
print(f"Levenshtein Distance:")
print(f"Average Edit Distance: {avg_lev:.2f}")

Levenshtein Distance:
Average Edit Distance: 2.38


In [20]:
# 5. Sentence Embedding Cosine Similarity (memory-efficient pairwise)
model = SentenceTransformer('all-MiniLM-L6-v2')
pred_embeds = model.encode(preds_l, convert_to_tensor=True, device='cuda')
ref_embeds = model.encode(refs_l, convert_to_tensor=True, device='cuda')

cos_sims = [
    util.cos_sim(pred_embeds[i], ref_embeds[i]).item()
    for i in range(len(preds_l))
]

print(f"\n🧠 Sentence-BERT Cosine Similarity:")
print(f"Average Cosine Similarity: {np.mean(cos_sims):.4f}")


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

Batches:   0%|          | 0/3145 [00:00<?, ?it/s]

Batches:   0%|          | 0/3145 [00:00<?, ?it/s]


🧠 Sentence-BERT Cosine Similarity:
Average Cosine Similarity: 0.7745
